<a href="https://colab.research.google.com/github/erwanBellon/Project-ML-SDM/blob/main/code/Mimicry_ANN_model_habitat_amount_3000_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part I: Setup



In [ ]:

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Cloning repo or fetch latest changes and path management
%cd /content
!rm -rf Project-ML-SDM
!git clone https://github.com/erwanBellon/Project-ML-SDM.git
%cd /content/Project-ML-SDM
!git pull

import os
from pathlib import Path

# Move into the project directory
%cd /content/2025_ML_EES/project/code
print("Current working directory:", Path.cwd())

# Define main project dir and outputs
PROJECT_ROOT_DIR = Path.cwd().parent       # -> /content/2025_ML_EES/project
OUTPUTS_PATH = PROJECT_ROOT_DIR / "outputs"
OUTPUTS_PATH.mkdir(parents=True, exist_ok=True)
print("Outputs will be saved to:", OUTPUTS_PATH)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, Input
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU detected. CNNs can be slow without GPU.")

# Common imports
import pandas as pd
import numpy as np
!pip install rasterio
import rasterio
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


# To make notebook reproducible
np.random.seed(123)
tf.random.set_seed(123)

# For plots
import matplotlib.pyplot as plt
%matplotlib inline

# Load Tensorboard
%load_ext tensorboard

Cloning into '2025_ML_EES'...
remote: Enumerating objects: 1048, done.
remote: Counting objects: 100% (206/206), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 1048 (delta 162), reused 52 (delta 52), pack-reused 842 (from 3)
Receiving objects: 100% (1048/1048), 256.92 MiB | 13.73 MiB/s, done.
Resolving deltas: 100% (682/682), done.
Updating files: 100% (368/368), done.
/content/2025_ML_EES
Already up to date.
/content/2025_ML_EES/project/code
Current working directory: /content/2025_ML_EES/project/code
Outputs will be saved to: /content/2025_ML_EES/project/outputs
No GPU detected. CNNs can be slow without GPU.


In [ ]:
print(Path.cwd())

/content/2025_ML_EES/project/code


# Part 2: Load files
## 2.1: Load the presences and absences landcover crops

In [ ]:
# --- Load table data ---
rds_path = Path("../data/Table_preds/function_3_100.rds")
!pip install pyreadr
import pyreadr
result = pyreadr.read_r(rds_path)
table_df = result[None]

# Select relevant columns
feature_df = table_df[['MAP','MAT','habitat_amount_3000']].astype(float)

# Labels come from presence/absence index lists
# Load image index files (no images) (just to get the indices that I need from my datatable)
presences_path = Path("../data/cropped_landcover/presences")
absences_path = Path("../data/cropped_landcover/absences")

def extract_indices(folder):
    tif_files = list(folder.glob("*.tif"))
    indices = []
    for tif in tif_files:
        idx = int(tif.stem.split("_")[-1])
        indices.append(idx)
    return np.array(indices)

indices_pres = extract_indices(presences_path)
indices_abs = extract_indices(absences_path)

# Build label vector
y = np.concatenate([
    np.ones(len(indices_pres)),
    np.zeros(len(indices_abs))
]).astype(np.float32)

# Extract corresponding rows
feature_all = feature_df
features = feature_all.iloc[np.concatenate([indices_pres, indices_abs])]
features = features.reset_index(drop=True)

# Normalize: no need to normalize after to avoid dataleakage
#table_features = (table_features - table_features.min()) / (table_features.max() - table_features.min())
#table_features = table_features.to_numpy(dtype=np.float32)

# Train/valid/test split
X_train, X_temp, y_train, y_temp = train_test_split(
    features, y, test_size=0.2, random_state=123, stratify=y
)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=123, stratify=y_temp
)


# Normalize using TRAIN ONLY

min_vals = X_train.min(axis=0)
max_vals = X_train.max(axis=0)

X_train = (X_train - min_vals) / (max_vals - min_vals)
X_valid = (X_valid - min_vals) / (max_vals - min_vals)
X_test  = (X_test  - min_vals) / (max_vals - min_vals)




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.2/776.2 kB 43.3 MB/s eta 0:00:00


## 2.2 Preprocess the data

In [ ]:
BATCH_SIZE = 8

def preprocess_table(table, label):
    table = tf.cast(table, tf.float32)
    label = tf.cast(label, tf.float32)
    return table, label

train_ds = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
      .shuffle(1000)
      .map(preprocess_table)
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE)
)

valid_ds = (
    tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
      .map(preprocess_table)
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE)
)

test_ds = (
    tf.data.Dataset.from_tensor_slices((X_test, y_test))
      .map(preprocess_table)
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE)
)

train_ds.name = "Training"
valid_ds.name = "Validation"
test_ds.name = "Test"


# 3. ANN model Training

In [ ]:
# Image branch: replace this branch by the proportion of forest

# Table branch
table_input = keras.Input(shape=(3,), name="table_input")
y_branch = layers.Dense(16, activation="relu")(table_input)
y_branch = layers.Dense(8, activation="relu")(y_branch)

#Previously CONCAT branch
z = layers.Dense(128, activation="relu")(y_branch)
z = layers.Dropout(0.4)(z)
z = layers.Dense(8, activation="relu")(z)
final_output = layers.Dense(1, activation="sigmoid", name="suitability")(z)

model = keras.Model(inputs=table_input, outputs=final_output)

model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC(name="auc")]
    )

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ table_input (InputLayer)        │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │         1,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ suitability (Dense)             │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,393 (9.35 KB)

 Trainable params: 2,393 (9.35 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
stop_early = keras.callbacks.EarlyStopping(
    monitor="val_auc", patience=10, mode="max", restore_best_weights=True
)

history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=50,
    callbacks=[stop_early]
)

Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - auc: 0.5399 - loss: 0.6904 - val_auc: 0.7521 - val_loss: 0.6785
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - auc: 0.6032 - loss: 0.6738 - val_auc: 0.7833 - val_loss: 0.6582
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - auc: 0.7258 - loss: 0.6391 - val_auc: 0.8167 - val_loss: 0.6218
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - auc: 0.7767 - loss: 0.6031 - val_auc: 0.8271 - val_loss: 0.5801
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - auc: 0.7763 - loss: 0.5405 - val_auc: 0.8021 - val_loss: 0.5552
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - auc: 0.7612 - loss: 0.5597 - val_auc: 0.8229 - val_loss: 0.5350
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - auc: 0.8100 - loss: 0.5167 - val_auc: 0.8229 - val_loss: 0.5291
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - auc: 0.7836 - loss: 0.5488 - val_auc: 0.8208 - val_loss: 0.5273
Epoch 9/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - auc: 0.8250 - los

In [ ]:
test_loss, test_auc = model.evaluate(test_ds)
print("\n Final Test AUC =", test_auc)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - auc: 0.7645 - loss: 0.6028

 Final Test AUC = 0.7791666984558105


MaxTSS is not implemented in keras. I'll compute
it there:

In [ ]:
def compute_maxTSS(y_true, y_prob):
    thresholds = np.linspace(0, 1, 200)
    best_tss = -2
    best_threshold = None

    for t in thresholds:
        preds = (y_prob >= t).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, preds).ravel()

        sensitivity = tp / (tp + fn + 1e-9)
        specificity = tn / (tn + fp + 1e-9)
        tss = sensitivity + specificity - 1

        if tss > best_tss:
            best_tss = tss
            best_threshold = t

    return best_tss, best_threshold


In [ ]:
y_prob = model.predict(test_ds)
y_true = np.concatenate([y for (x, y) in test_ds])
max_tss, best_threshold = compute_maxTSS(y_true, y_prob)

print("MaxTSS:", max_tss)
print("Optimal threshold:", best_threshold)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
MaxTSS: 0.5333333332372221
Optimal threshold: 0.4472361809045226


Save my best model and mannually load it in GitHub

In [ ]:
from google.colab import files
model.save("Mirror_ANN_HA_3000_bestModel.keras")
files.download("Mirror_ANN_HA_3000_bestModel.keras")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>